# Generación de Imágenes de Mangos usando Redes Generativas Adversarial (GANs)



## Introducción
En este notebook se entrena una Red Generativa Adversarial (GAN) para generar imágenes sintéticas de mangos a partir de un dataset de imágenes reales.

Las GANs (Generative Adversarial Networks), introducidas por Ian Goodfellow en 2014, constan de dos redes neuronales: el **generador**, que intenta crear imágenes realistas, y el **discriminador**, que intenta distinguir entre imágenes reales y generadas. Ambas redes se entrenan de forma simultánea y competitiva, mejorando mutuamente.

Este enfoque permite la generación de datos visuales convincentes, útil en aplicaciones como aumento de datos, arte generativo y simulación.

## Instalación de dependencias

In [ ]:
#Ejecutar esta linea en caso de presentar dificultades de versiones
#!pip install numpy==1.26 tensorflow==2.14  tensorflow-addons==0.22.0 matplotlib keras==2.14

In [ ]:
!pip install tensorflow matplotlib keras numpy

Montar google drive para acceder a la base de datos

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Importación de librerias

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt


## Configuración de hiperparámetros

In [ ]:

# =================== CONFIG ===================
IMAGE_SIZE = 64
LATENT_DIM = 100
BATCH_SIZE = 64
EPOCHS = 3000
SAVE_INTERVAL = 500
DATA_DIR = '/content/drive/MyDrive/db_320x320'  # <-- CAMBIA ESTO A TU CARPETA
OUTPUT_DIR = '/content/drive/MyDrive/mangos_generados'
os.makedirs(OUTPUT_DIR, exist_ok=True)

## Funciones de importación de imagenes para el entrenamiento y exportación de los resultados.

In [ ]:

# =================== CARGAR IMÁGENES ===================
def load_images_from_folder(folder, size=(IMAGE_SIZE, IMAGE_SIZE)):
    images = []
    for filename in os.listdir(folder):
        path = os.path.join(folder, filename)
        try:
            img = load_img(path, target_size=size)  # RGB por defecto
            img = img_to_array(img)
            img = (img / 127.5) - 1.0  # Normalizar a [-1, 1]
            images.append(img)
        except:
            continue
    return np.array(images)

In [ ]:
# =================== GUARDAR IMÁGENES ===================
def save_best_images(images, epoch, discriminator, top_k=3):
    """
    Guarda solo las `top_k` mejores imágenes según la puntuación del discriminador.
    """
    # Desnormalizar imágenes para guardar
    images = (images * 127.5 + 127.5).numpy().astype(np.uint8)

    # Evaluar imágenes con el discriminador
    images_input = tf.convert_to_tensor((images / 127.5) - 1.0, dtype=tf.float32)  # Volver a [-1, 1]
    scores = discriminator(images_input, training=False).numpy().flatten()

    # Obtener los índices de las mejores imágenes
    best_indices = np.argsort(-scores)[:top_k]  # Orden descendente

    for i, idx in enumerate(best_indices):
        img = images[idx]
        plt.imsave(f"{OUTPUT_DIR}/epoch{epoch}_top{i+1}.png", img)


## Definición del Generador

El generador es una red neuronal que toma como entrada un vector aleatorio (ruido) y lo transforma en una imagen sintética. Su objetivo es aprender a generar imágenes que el discriminador no pueda diferenciar de las reales.

Generalmente incluye capas de expansión (Dense), normalización (BatchNormalization), y reescalado (UpSampling2D o Conv2DTranspose) para formar imágenes a partir del ruido.

In [ ]:

# =================== DEFINIR GENERADOR ===================
def build_generator():
    model = tf.keras.Sequential([
        layers.Dense(8 * 8 * 256, use_bias=False, input_shape=(LATENT_DIM,)),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Reshape((8, 8, 256)),

        layers.Conv2DTranspose(128, 5, strides=2, padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        layers.Conv2DTranspose(64, 5, strides=2, padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        layers.Conv2DTranspose(3, 5, strides=2, padding='same', activation='tanh', use_bias=False)
    ])
    return model


## Definición del Discriminador

El discriminador es una red neuronal convolucional que toma como entrada una imagen (real o generada) y predice si es auténtica o falsa. Funciona como un clasificador binario y su objetivo es mejorar su capacidad de detección durante el entrenamiento.

Contiene capas Conv2D, activaciones LeakyReLU, y Dropout para evitar el sobreajuste.


In [ ]:
# =================== DEFINIR DISCRIMINADOR ===================
def build_discriminator():
    model = tf.keras.Sequential([
        layers.Conv2D(64, 5, strides=2, padding='same', input_shape=[64, 64, 3]),
        layers.LeakyReLU(),
        layers.Dropout(0.3),

        layers.Conv2D(128, 5, strides=2, padding='same'),
        layers.LeakyReLU(),
        layers.Dropout(0.3),

        layers.Conv2D(256, 5, strides=2, padding='same'),
        layers.LeakyReLU(),
        layers.Dropout(0.3),

        layers.Flatten(),
        layers.Dense(1)
    ])
    return model


## Compilación y Función de Pérdida

Ambos modelos son entrenados en conjunto, donde la función de pérdida del generador busca engañar al discriminador, y la del discriminador intenta distinguir correctamente las imágenes reales de las falsas. Se usan funciones como `binary_crossentropy` y optimizadores como Adam.


## Proceso de Entrenamiento
Durante cada época del entrenamiento:

- Se generan vectores de ruido y se pasan por el generador para crear imágenes falsas.
- Se seleccionan imágenes reales del dataset.
- El discriminador se entrena con ambas (reales y falsas).
- Luego, se entrena el generador para mejorar su capacidad de engañar al discriminador.

Este proceso competitivo se repite por múltiples épocas, mejorando la calidad de las imágenes generadas con el tiempo.

In [ ]:
# =================== ENTRENAMIENTO ===================
def train():
    print("Cargando imágenes...")
    images = load_images_from_folder(DATA_DIR)
    dataset = tf.data.Dataset.from_tensor_slices(images).shuffle(1000).batch(BATCH_SIZE)
    print(f"{len(images)} imágenes cargadas.")

    generator = build_generator()
    discriminator = build_discriminator()

    cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

    generator_optimizer = tf.keras.optimizers.Adam(1e-4)
    discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

    @tf.function
    def train_step(images):
        noise = tf.random.normal([BATCH_SIZE, LATENT_DIM])

        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            generated_images = generator(noise, training=True)

            real_output = discriminator(images, training=True)
            fake_output = discriminator(generated_images, training=True)

            gen_loss = cross_entropy(tf.ones_like(fake_output), fake_output)
            disc_loss = cross_entropy(tf.ones_like(real_output), real_output) + \
                        cross_entropy(tf.zeros_like(fake_output), fake_output)

        gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

        generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
        discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

        return gen_loss, disc_loss

    print("Iniciando entrenamiento...")
    for epoch in range(EPOCHS):
        print(f"Epoch {epoch+1}/{EPOCHS}")
        for image_batch in dataset:
            gen_loss, disc_loss = train_step(image_batch)

        if (epoch + 1) % SAVE_INTERVAL == 0:
            print(f"Epoch {epoch+1}, Gen Loss: {gen_loss.numpy():.4f}, Disc Loss: {disc_loss.numpy():.4f}")
            noise = tf.random.normal([16, LATENT_DIM])
            generated_images = generator(noise, training=False)
            generated_images = (generated_images + 1) / 2
            save_best_images(generated_images, epoch + 1, discriminator)


    print("Entrenamiento finalizado.")

# =================== MAIN ===================
train()

Se han truncado las últimas 5000 líneas del flujo de salida.
Epoch 3037/10000
Epoch 3038/10000
Epoch 3039/10000
Epoch 3040/10000
Epoch 3041/10000
Epoch 3042/10000
Epoch 3043/10000
Epoch 3044/10000
Epoch 3045/10000
Epoch 3046/10000
Epoch 3047/10000
Epoch 3048/10000
Epoch 3049/10000
Epoch 3050/10000
Epoch 3051/10000
Epoch 3052/10000
Epoch 3053/10000
Epoch 3054/10000
Epoch 3055/10000
Epoch 3056/10000
Epoch 3057/10000
Epoch 3058/10000
Epoch 3059/10000
Epoch 3060/10000
Epoch 3061/10000
Epoch 3062/10000
Epoch 3063/10000
Epoch 3064/10000
Epoch 3065/10000
Epoch 3066/10000
Epoch 3067/10000
Epoch 3068/10000
Epoch 3069/10000
Epoch 3070/10000
Epoch 3071/10000
Epoch 3072/10000
Epoch 3073/10000
Epoch 3074/10000
Epoch 3075/10000
Epoch 3076/10000
Epoch 3077/10000
Epoch 3078/10000
Epoch 3079/10000
Epoch 3080/10000
Epoch 3081/10000
Epoch 3082/10000
Epoch 3083/10000
Epoch 3084/10000
Epoch 3085/10000
Epoch 3086/10000
Epoch 3087/10000
Epoch 3088/10000
Epoch 3089/10000
Epoch 3090/10000
Epoch 3091/10000
Epoc

KeyboardInterrupt: 


## Conclusión

El entrenamiento de una GAN permite generar imágenes sintéticas convincentes a partir de ruido. En este caso, hemos entrenado una red capaz de crear imágenes que imitan mangos reales.

Aunque las primeras épocas muestran imágenes borrosas o irreconocibles, el generador mejora conforme avanza el entrenamiento, aprendiendo las características visuales esenciales del conjunto de entrenamiento.

Este enfoque puede extenderse a otras frutas o elementos visuales, y representa un paso fundamental hacia la síntesis automática de imágenes en el campo de la inteligencia artificial generativa.